In [ ]:
!pip install power-grid-model-ds[visualizer] --quiet

# ⚡ Advanced Power Grid Workshop: Solving an Overload Scenario

You're a senior grid analyst at GridNova Utilities, overseeing a legacy radial distribution network in a semi-rural region. Recently, customer load growth has increased dramatically, particularly in areas served by a single long feeder. This has pushed some branches past their capacity, triggering repeated overloads.

Your task: augment the grid by adding a second substation and relieving the overloaded feeder through targeted switching.

This hands-on simulation walks you through each step of diagnosing, planning, and solving this overload using the Power Grid Model DS library.

## 🎯 Workshop Goals
- Detect a line overload after load increase.
- Find a suitable node to connect a new substation.
- Trace the overloaded route.
- Strategically open a line to reroute power and relieve the feeder.



In [ ]:
import numpy as np
from dataclasses import dataclass
from power_grid_model_ds import Grid, GraphContainer
from power_grid_model_ds.arrays import NodeArray, LineArray, SourceArray
from power_grid_model_ds.enums import NodeType

# 🧪 Step 1: Extend the Data Model
Goal: Add coordinate fields and tracking for simulated voltages and line currents.

You’ll subclass NodeArray and LineArray to add:

- x, y coordinates for spatial logic and plotting
- u for node voltage results
- i_from for line currents
- A computed .is_overloaded property for easy filtering

In [ ]:
# 📦 Extend the grid with x, y, u (node) and i_from (line)
from numpy.typing import NDArray

class MyNodeArray(NodeArray):
    _defaults = {"x": 0.0, "y": 0.0, "u": 0.0}
    x: NDArray[np.float64]
    y: NDArray[np.float64]
    u: NDArray[np.float64]

class MyLineArray(LineArray):
    _defaults = {"i_from": 0.0}
    i_from: NDArray[np.float64]

    @property
    def is_overloaded(self) -> NDArray[np.bool_]:
        """Check if the line is overloaded."""
        return self.i_from > self.i_n

@dataclass
class MyGrid(Grid):
    node: MyNodeArray
    line: MyLineArray
    graphs: GraphContainer

# 🏗️ Step 2: Generate and Prepare the Grid
Goal: Build a synthetic radial grid with randomized node positions and realistic load levels.

You’ll use RadialGridGenerator to build a 20-node grid and then:

- Randomize node coordinates in a 1000×1000 space
- Assign loads to simulate high demand
- You’ll then compute a power flow to get realistic line currents.

In [ ]:
from power_grid_model_ds.generators import RadialGridGenerator

# Generate a radial grid
grid_gen = RadialGridGenerator(grid_class=MyGrid, nr_nodes=20, nr_sources=1)
grid = grid_gen.run(seed=42)

np.random.seed(42)
num_nodes = grid.node.size
x_coords = np.random.randint(0, 1000, size=num_nodes)
y_coords = np.random.randint(0, 1000, size=num_nodes)

grid.node.x = x_coords
grid.node.y = y_coords

In [ ]:
grid.sym_load.p_specified = np.random.randint(300000, 1200000, size=grid.sym_load.size)

In [ ]:
grid.set_feeder_ids()
print(grid.node)

# 🧯 Step 3: Detect the Overload
Goal: Identify which line(s) are exceeding their rated current.

You’ll:

- Use the .is_overloaded property to find the problem
- Isolate the feeder route to the affected line

This gives a clear target for where you need to intervene.

In [ ]:
# Check the grid for capacity issues
# 1. Use the PowerGridModelInterface to calculate power flow
# 2. Update the grid with the calculated values
# 3. Return the lines (LineArray) that are overloaded

# Hint: You can use the `is_overloaded` property of the `MyLineArray` class to check for overloaded lines.
# Hint: https://power-grid-model-ds.readthedocs.io/en/stable/quick_start.html#performing-power-flow-calculations

def check_for_capacity_issues(grid: Grid) -> LineArray:
    """Check for capacity issues on the grid.
    Return the lines that with capacity issues.
    """

from power_grid_model_ds import PowerGridModelInterface

def check_for_capacity_issues(grid: Grid) -> LineArray:
    """Check for capacity issues on the grid.
    Return the lines that with capacity issues.
    """
    pgm_interface = PowerGridModelInterface(grid)
    pgm_interface.calculate_power_flow()
    pgm_interface.update_grid()

    # update attribute overload_status

    return grid.line[grid.line.is_overloaded]

print(check_for_capacity_issues(grid))

# 🧭 Step 4: Plan a Relief Strategy
Goal: Place a second substation near the overloaded path.

You’ll:
- Add a new substation at a specified location
- Connect it to the closest node in the network
- Visually inspect the grid with visualize()

This substation will act as a new injection point for rerouting load.


In [ ]:
def build_new_substation(grid: Grid, location: tuple[float, float]) -> NodeArray:
    """Build a new substation at the given location.
    Return the new substation.
    """
    new_substation = MyNodeArray(
        node_type=[NodeType.SUBSTATION_NODE.value],
        u_rated=[10500.0],
        x=[location[0]],
        y=[location[1]]
    )
    grid.append(new_substation)

    new_source = SourceArray(
        node=[new_substation.id.item()],
        status=[1],
        u_ref=[1],
    )
    grid.append(new_source)
    return new_substation

# 🏗️ Add a new substation at (212, 133)
new_substation = build_new_substation(grid, (212, 133))

In [ ]:
# visualize the grid
from power_grid_model_ds.visualizer import visualize
# visualize(grid)

# 🔗 Step 5: Trace and Analyze the Overloaded Route
Goal: Identify a switchable line along the route from the new node to the old substation.

You’ll:
- Use the graph interface to trace the shortest path from the new substation to the original
- Calculate cumulative load along the path
- Find a cut point where the remaining load toward the old substation is < 2 MW

This ensures you relieve the feeder while maintaining supply continuity.

In [ ]:
def get_all_congested_routes(grid: Grid) -> list[NodeArray]:
    """Get all routes that originate from a given substation node."""
    grid.set_feeder_ids()
    lines_with_congestion = check_for_capacity_issues(grid)
    feeder_branch_ids_with_congestion = np.unique(lines_with_congestion['feeder_branch_id'])
    return [grid.node.filter(feeder_branch_id=branch_id) for branch_id in feeder_branch_ids_with_congestion]
 
congested_routes = get_all_congested_routes(grid)

In [ ]:
def find_connection_point(route: NodeArray, new_substation: NodeArray) -> NodeArray:
    """Calculate the connection point for the new route.
    This should be the geographically closest node to the new substation.
    """
    x_difference = route.x - new_substation.x
    y_difference = route.y - new_substation.y
    distances = (x_difference**2 + y_difference**2) ** 0.5
    
    idx_closest_node = np.argmin(distances)
    closest_node = grid.node[idx_closest_node]
    return closest_node


def connect_to_route(grid: Grid, connection_point: NodeArray, new_substation: NodeArray) -> None:
    """Connect the new substation node to the connection point.
    """
    # Create a new line that connects the two nodes
    new_line = MyLineArray(
        from_node=[new_substation.id],
        to_node=[connection_point.id],
        from_status=[0], # status is 0 to make sure the line is not active
        to_status=[1],
        i_n=[200.0],
        r1=[0.05], x1=[0.01], c1=[0.0], tan1=[0.0]
    )
    grid.append(new_line)

# ✂️ Step 6: Open the Right Line
Goal: Deactivate a line to offload the original feeder.

You’ll:
- Identify and deactivate the line between two nodes on the critical path
- Rerun power flow after the switch
- Confirm the overload is resolved

This final step demonstrates how network topology and load can be managed dynamically with switching and distributed generation.



In [ ]:
def optimize_route_transfer(grid: Grid, connection_point: NodeArray, new_substation: NodeArray) -> None:
    """Attempt to optimize the route transfer moving the naturally open point (NOP) upstream towards the old substation.
    This way, the new substation will take over more nodes of the original route.

    Note that a node cannot be taken over if that results in a capacity issue on the grid.
    """
    # 🔗 Trace path from that node to the current (old) substation
    old_substation_node_id = connection_point.feeder_node_id.item()
    path, length = grid.graphs.active_graph.get_shortest_path(connection_point.id.item(), old_substation_node_id)
    print("Path from overload to old substation:", path)

    current_branch = grid.line.filter(
        from_node=[connection_point.id.item(), new_substation.id.item()],
        to_node=[connection_point.id.item(), new_substation.id.item()]
    )
    for i, (from_node, to_node) in enumerate(zip(path[0:-1], path[1:])):
        # Check if the line is overloaded
        if not any(check_for_capacity_issues(grid)):
            break

        grid.make_active(current_branch)
        current_branch = grid.line.filter(from_node=[from_node, to_node], to_node=[from_node, to_node])
        grid.make_inactive(current_branch)

In [ ]:
def transfer_routes(grid: Grid, new_substation: NodeArray) -> NodeArray:
    """Migrate a subset of the routes of the old substation to the new substation.
    Each route can be migrated fully or partially.

    """
    congested_routes = get_all_congested_routes(grid)

    for route in congested_routes:
        closest_node = find_connection_point(
            route=route,
            new_substation=new_substation
        )

        connect_to_route(
            grid=grid,
            connection_point=closest_node,
            new_substation=new_substation,
        )

        optimize_route_transfer(
            grid=grid,
            connection_point=closest_node,
            new_substation=new_substation)
        
        print(f"Connected new substation to node {closest_node.id}")

transfer_routes(grid=grid, new_substation=new_substation)

In [ ]:
print(check_for_capacity_issues(grid))

# ✅ Wrap-Up
You’ve just:

Diagnosed a power system constraint
- Planned and executed a grid topology change
- Verified success with power flow simulations

We hope you enjoyed working with Power Grid Model DS and would love to hear your feedback